# Introduction to Natural Language Processing (NLP)


_Summarized by QH_  
_First version: 2023-07-16_  
_Last updated on : 2023-07-23_  

## What is NLP?
It is a field of using statistics and computers to understand languages. It can help to identify topic, classify text.

We can use NLP in the following applications:
* Chatbots
* Translation
* Sentiment analysis


## Preparation for NLP

### Data Prepocessing
#### Tokenization
Tokens are smaller chunks of text from a string or document which can help us understand better of the text (e.g., excluding unwanted or uninformative chunks).

The process of breaking into tokens is called _Tokenization_.
We can use regular expressions help with this task, e.g.:
* Breaking out words or sentences
* Separating punctuaion
* Separating all hashtags in a tweet.
#### Stemming and Lemmatization
Lemmatization: determining the root word.
Stemming: simpler version of lemmatization - stripping suffixes from the end of the word.

#### Sentence segmentation
Sentence segmentation: breaking up a text into individual sentences, using cues like perios or exclamation points.

#### Stop word removal


There are several python packages that helps with tokenization.
* `re`
* `tokenize` from `nltk`
    * `word_tokenize`: tokenize a document into words.
    * `sent_tokenize`: tokenize a document into sentences.
    * `regexp_tokenize`: tokenize a string or document based on a regular expression pattern
    * `TweetTokenizer`: special class just for tweet tokenization, allowing you to separate hashtags, mentions and lots of exclamation points

### Feature extraction

#### Bag-of-words (BOW)
tba.
#### Term Frequency - Inverse Document Frequency (TF-IDF)
tba.
#### Word2Vec
tba.
#### GLoVE
tba.